#### Submitted By: Lior Kricheli, Eliran Malka and Itamar Yacoby

In [2]:
# TensorFlow and tf.keras
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

import seaborn as sns

print(tf.__version__)

2.3.0


In [3]:
ls archive\Kather_texture_2016_image_tiles_5000\Kather_texture_2016_image_tiles_5000

 Volume in drive C is Acer
 Volume Serial Number is 96D3-27E0

 Directory of C:\Users\Itamar\Desktop\DeepLearningFinalProject\archive\Kather_texture_2016_image_tiles_5000\Kather_texture_2016_image_tiles_5000

06/03/2021  11:05 AM    <DIR>          .
06/03/2021  11:05 AM    <DIR>          ..
06/03/2021  11:05 AM    <DIR>          train
06/03/2021  11:06 AM    <DIR>          validation
               0 File(s)              0 bytes
               4 Dir(s)  363,133,812,736 bytes free


In [4]:
pathTrain = 'archive\\Kather_texture_2016_image_tiles_5000\\Kather_texture_2016_image_tiles_5000\\train'
pathTest= 'archive\\Kather_texture_2016_image_tiles_5000\\Kather_texture_2016_image_tiles_5000\\validation'
my_batch_size = 32

dataGen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = dataGen.flow_from_directory(pathTrain, 
                                               
                                              batch_size=my_batch_size, 
                                              class_mode='categorical')

dataGenTest = ImageDataGenerator(rescale=1./255)
test_generator = dataGen.flow_from_directory(pathTest, 
                                              
                                             batch_size=my_batch_size, 
                                             class_mode='categorical', 
                                             shuffle=False)

Found 5000 images belonging to 8 classes.
Found 496 images belonging to 8 classes.


In [34]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

model.compile(optimizer='sgd', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 37, 37, 128)      

In [35]:
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples/train_generator.batch_size,
                    epochs=25,
                    validation_data=test_generator,
                    verbose=1)

Epoch 1/25


InvalidArgumentError:  Input to reshape is a tensor with 4194304 values, but the requested shape requires a multiple of 41472
	 [[node sequential_3/flatten_2/Reshape (defined at <ipython-input-35-c00df31bbe7e>:1) ]] [Op:__inference_train_function_2514]

Function call stack:
train_function
